- **수집한 데이터에서 필요한 데이터**: 이닝, 투수, 볼카운트, 결과, 아웃카운트상황, 주자상황
- **삭제해야 하는 데이터**: 결과가 4구, 사구, 희생번트, 희생플라이
- **필요한 데이터**
    - **이닝**: 초, 말 둘다 같은 회로 처리해야함
    - **투수**: 그냥 이름으로 처리
    - **볼카운트**: 괄호 안에 있는 것만 (볼-스트라이크)
    - **결과**: 1- 안타 0- 노안타
    - **아웃카운트상황**: 무사, 1사, 2사
    - **주자상황**: 아무것도 안적혀 있으면-주자없음, 1루, 2루, 3루, 1,2루, 1,3루, 2,3루, 만루

- **따라서 데이터에서 필요한 인덱스**: Inning, Pitcher, BC, Result, Prev(아웃카운트 상황, 주자상황)

1. 이닝 전처리
   - 데이터 불러오기
   - 필요한 열만 남겨두기
   - Inning 인덱스 불러오기
   - n회 말, 초 => 모두 n회로 처리

In [ ]:
import pandas as pd
import re
import schedule
import time

def preprocess_data(file_path, name):
    df = pd.read_csv(file_path)
    
    # Keep only the required columns
    columns_to_keep = ['Inning', 'Pitcher', 'Result', 'Prev']
    df_filtered = df[columns_to_keep]

    # Step 1: Inning 전처리
    df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")
    print("이닝 전처리 완료")
    
    # Step 2: 불필요한 행 제거 (Result 열에서 특정 문자열 포함하는 행)
    # 제거할 조건 정의
    remove_conditions = ['4구', '사구', '희생']
    
    # 제거 조건에 해당하는 행 제거
    for condition in remove_conditions:
        df_filtered = df_filtered[~df_filtered['Result'].str.contains(condition, case=False, na=False)]
    print("불필요한 행 제거 완료")
    
    # Step 3: Result 전처리
    # 히트 조건 정의
    hit_conditions = ['안타', '1루타', '2루타', '3루타', '홈런']
    
    # Result 열을 1과 0으로 변환
    df_filtered['Result'] = df_filtered['Result'].apply(lambda x: 1 if any(hit in x.lower() for hit in hit_conditions) else 0)
    
    print("결과 전처리 완료")
    
    # Step 4: Prev 전처리 (n:n 형식의 점수 제거)
    df_filtered['Prev'] = df_filtered['Prev'].str.replace(r'\b\d+:\d+\b', '', regex=True).str.strip()
    
    print("Prev 전처리 완료")
    
    # Step 5: Prev 열을 Out과 Bases로 분리
    def split_prev_column(text):
        # 정규 표현식을 사용하여 "x사"와 "x,x루" 부분을 분리
        match = re.search(r'(무사|\d사)\s*(.*루)?', text)
        if match:
            out, bases = match.groups()
            if bases is None or bases.strip() == '':
                bases = '주자없음'
            return pd.Series([out, bases])
        return pd.Series(['', ''])  # 매칭이 되지 않는 경우 빈 문자열 반환
    
    df_filtered[['Out', 'Bases']] = df_filtered['Prev'].apply(split_prev_column)
    
    # Prev 열 삭제
    df_filtered = df_filtered.drop(columns=['Prev'])
    
    print("Prev 열 분리 완료")
    
    # Step 6: Result 열을 가장 오른쪽으로 이동
    cols = [col for col in df_filtered.columns if col != 'Result'] + ['Result']
    df_filtered = df_filtered[cols]
    
    print("Result 열 이동 완료")
    
    # 전처리된 데이터를 CSV 파일로 저장
    output_file_path = './project_kbo/preprocess/'+name+'_preprocessed.csv'
    df_filtered.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    print(f"데이터 전처리 완료 {output_file_path}")

def schedule_preprocessing():
    players = [
        '강민호', '구자욱', '김영웅', '김재혁', '김지찬', '김헌곤',
        '김현준', '류지혁', '박병호', '안주형', '윤정빈', '이병헌',
        '이성규', '이재현', '전병우'
    ]

    for player in players:
        file_path = f'./project_kbo/data/{player}_2024.csv'
        schedule.every().day.at("10:33").do(preprocess_data, file_path, player)

    print("모든 선수에 대한 스케줄링이 완료되었습니다.")

# 스케줄링 함수 호출
schedule_preprocessing()

# 스케줄링된 작업을 실행
while True:
    schedule.run_pending()
    time.sleep(60)

모든 선수에 대한 스케줄링이 완료되었습니다.


C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")
C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")
C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyW

이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/강민호_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/구자욱_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/김영웅_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/김재혁_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/김지찬_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/김헌곤_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/김현준_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./proj

C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")
C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")
C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyW

불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/이재현_preprocessed.csv
이닝 전처리 완료
불필요한 행 제거 완료
결과 전처리 완료
Prev 전처리 완료
Prev 열 분리 완료
Result 열 이동 완료
데이터 전처리 완료 ./project_kbo/preprocess/전병우_preprocessed.csv


C:\Users\rhee hyun seo\AppData\Local\Temp\ipykernel_59068\3238412097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Inning'] = df_filtered['Inning'].str.replace("말", "").str.replace("초", "")


타자들 전처리 완료